In [1]:
from pathlib import Path
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_path = Path("../data/curated/water_consumption_curated.parquet")
df = pd.read_parquet(df_path)
df.head()

,id,timestamp,second,minute,hour,day,weekday,week_of_year,month,year,time_passed_seconds,input_flow_rate,reservoir_level_percentage,reservoir_level_liters,total_liters_entered,effective_liters_entered,total_liters_out,output_flow_rate,pressure,pump_1,pump_2,total_precip_mm,station_pressure_mb,max_pressure_last_hour_mb,min_pressure_last_hour_mb,global_radiation_kj_m2,air_temp_c,dew_point_temp_c,max_temp_last_hour_c,min_temp_last_hour_c,max_dew_point_last_hour_c,min_dew_point_last_hour_c,max_humidity_last_hour_percentage,min_humidity_last_hour_percentage,relative_humidity_percentage,wind_direction_deg,max_wind_gust_m_s,wind_speed_m_s
0,1,2023-03-17 12:28:56,56,28,12,17,4,11,3,2023,3710.0,66.05,35.86,358600.0,245045.5,60000.0,185045.5,49.88,38.20,0,1,0.0,1015.1,1015.1,1014.6,1413.7,27.9,22.0,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
1,2,2023-03-17 12:31:26,26,31,12,17,4,11,3,2023,150.0,65.64,36.16,361600.0,9846.0,3000.0,6846.0,45.64,38.06,0,1,0.0,1015.1,1015.1,1014.6,1413.7,27.9,22.0,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
2,3,2023-03-17 12:33:56,56,33,12,17,4,11,3,2023,150.0,65.64,36.50,365000.0,9846.0,3400.0,6446.0,42.97,38.03,0,1,0.0,1015.1,1015.1,1014.6,1413.7,27.9,22.0,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
3,4,2023-03-17 12:36:26,26,36,12,17,4,11,3,2023,150.0,65.64,36.80,368000.0,9846.0,3000.0,6846.0,45.64,38.17,0,1,0.0,1015.1,1015.1,1014.6,1413.7,27.9,22.0,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8
4,5,2023-03-17 12:38:56,56,38,12,17,4,11,3,2023,150.0,65.24,36.80,368000.0,9786.0,0.0,9786.0,65.24,38.17,0,1,0.0,1015.1,1015.1,1014.6,1413.7,27.9,22.0,27.9,25.9,22.9,20.8,82.0,66.0,70.0,337.0,2.2,0.8


In [5]:
def get_average_flow_out_across_day(original_df: pd.DataFrame) -> pd.DataFrame:
    df = original_df.copy()
    
    # Separate data into weekdays and weekends
    df_weekdays = df[df['weekday'] < 5]
    df_weekends = df[df['weekday'] >= 5]

    # Calculate average flow out per hour for weekdays and weekends
    flow_out_weekdays = df_weekdays.groupby('hour')['output_flow_rate'].mean().reset_index().rename(columns={'output_flow_rate': 'avg_weekday_output_flow'})
    flow_out_weekends = df_weekends.groupby('hour')['output_flow_rate'].mean().reset_index().rename(columns={'output_flow_rate': 'avg_weekend_output_flow'})
    
    # Merge the weekday and weekend data into a single DataFrame
    df_combined = pd.merge(flow_out_weekdays, flow_out_weekends, on='hour', how='outer')
    
    return df_combined


In [6]:
average_flow_out_across_day = get_average_flow_out_across_day(df)
average_flow_out_across_day.head(24)

,hour,avg_weekday_output_flow,avg_weekend_output_flow
0,0,28.017858,30.630013
1,1,25.504243,27.019786
2,2,25.460058,25.009778
3,3,23.178671,24.169199
4,4,23.729474,24.319796
5,5,27.374432,27.062895
6,6,30.741613,28.136149
7,7,33.857773,30.997651
8,8,35.803129,34.450577
9,9,35.865222,39.713231
